In [7]:
# Used ChatGPT to teach me how to "fuzzy" match data in output HTML vs. 
# player data in the JSON output from API. Use these libraries!
pip install beautifulsoup4 fuzzywuzzy  python-Levenshtein


SyntaxError: invalid syntax (1556007417.py, line 3)

In [41]:
import http.client
import pprint
import json
import os
import requests
import time
import random
import sys
import shelve
import inspect
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()



True

In [38]:
# Global vars
LEAUGE_JSON = 'jleague_data.json'
IMAGE_DIR='images'


In [ ]:
# jleague_data.json has J League data, teams and their ids
if __name__ == "__main__":
    with open(LEAUGE_JSON, 'r') as read_file:
        league_data = json.load(read_file)
    
    pprint.pprint(league_data)

In [4]:
# We needed to explore the league_data to suss out how to extract individual teams.
# It may be a thought to make this a dictionary of some sort (but maybe not).
i = 0
for team in league_data['response']:
    print(i)  
    print(team)
    i += 1
  

0
{'team': {'id': 279, 'name': 'Consadole Sapporo', 'code': 'SAP', 'country': 'Japan', 'founded': 1935, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/279.png'}, 'venue': {'id': 955, 'name': 'Sapporo Dome', 'address': '1-3 Hitsujigaoka, Toyohira-ku', 'city': 'Sapporo', 'capacity': 53845, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/955.png'}}
1
{'team': {'id': 281, 'name': 'Kashiwa Reysol', 'code': 'KAS', 'country': 'Japan', 'founded': 1940, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/281.png'}, 'venue': {'id': 965, 'name': 'SANKYO FRONTIER Kashiwa Stadium', 'address': '1-2-50 Kashiwa, Chiba', 'city': 'Kashiwa', 'capacity': 15900, 'surface': 'grass', 'image': 'https://media-4.api-sports.io/football/venues/965.png'}}
2
{'team': {'id': 282, 'name': 'Sanfrecce Hiroshima', 'code': 'HIR', 'country': 'Japan', 'founded': 1938, 'national': False, 'logo': 'https://media-4.api-sports.io/football/teams/282.png'}

In [6]:
# We found Kawasaki Frontale with counting
kawasaki_frontale = league_data['response'][11]
pprint.pprint(kawasaki_frontale)

{'team': {'code': 'KAW',
          'country': 'Japan',
          'founded': 1955,
          'id': 294,
          'logo': 'https://media-4.api-sports.io/football/teams/294.png',
          'name': 'Kawasaki Frontale',
          'national': False},
 'venue': {'address': '1-1 Todoroki, Nakahara-ku',
           'capacity': 27495,
           'city': 'Kawasaki',
           'id': 966,
           'image': 'https://media-4.api-sports.io/football/venues/966.png',
           'name': 'Kawasaki Todoroki Stadium',
           'surface': 'grass'}}


In [5]:
# We use the boilerplate test code available to do simple API call.
# It may be possible to invoke `conn.request` if we create a 
# proper string with the squad_id. f-string interpolation in 
# conn.request itself seems to not be possible

# To get easy access to these environmental variables you must
# create a `.env` file that contains key/value pairs in this format:
# 
# xrapidapi-key=XXXXXXXXXXX
# KAWASAKI_SQUAD_ID=294
# 
# This repo places these in the .gitignore, since you *really* don't
# want to accidentally check in the API key you have into a publicly
# accessible version control system.

import http.client
conn = http.client.HTTPSConnection("v3.football.api-sports.io")
api_key = os.getenv('x-rapidapi-key')
squad_id = os.getenv('KAWASAKI_SQUAD_ID')
headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': api_key
    }

#print(f"X-RAPIDAPI-KEY:{api_key}")

# We tried being cheeky and specifying the team ID via f-string, but that 
# did not go as planned. 
conn.request("GET", "/players/squads?team=294", headers=headers)

res = conn.getresponse()
data = res.read()

squad_data = data.decode("utf-8")
#pprint(data.decode("utf-8"))
pprint.pprint(squad_data)
                         

TypeError: expected string or bytes-like object

In [22]:
# This cell is same as obove, but we were experimenting with persisting
# the data in local JSON, which we did via:
#
# squad_data_json = json.load(res)
#

import http.client
conn = http.client.HTTPSConnection("v3.football.api-sports.io")
api_key = os.getenv('x-rapidapi-key')
squad_id = os.getenv('KAWASAKI_SQUAD_ID')

#print(f"X-RAPIDAPI-KEY:{api_key}")


headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': api_key
    }


conn.request("GET", "/players/squads?team=294", headers=headers)

res = conn.getresponse()
#data = res.read()

# Persist the squad data in JSON so we don't take API hits unneccessarily
squad_data_json = json.load(res)
#pprint(data.decode("utf-8"))
pprint.pprint(squad_data_json)
                         

{'errors': [],
 'get': 'players/squads',
 'paging': {'current': 1, 'total': 1},
 'parameters': {'team': '294'},
 'response': [{'players': [{'age': 38,
                            'id': 32946,
                            'name': 'Jung Sung-Ryong',
                            'number': 1,
                            'photo': 'https://media-4.api-sports.io/football/players/32946.png',
                            'position': 'Goalkeeper'},
                           {'age': 33,
                            'id': 32943,
                            'name': 'S. Ando',
                            'number': 21,
                            'photo': 'https://media-4.api-sports.io/football/players/32943.png',
                            'position': 'Goalkeeper'},
                           {'age': 24,
                            'id': 315353,
                            'name': 'Y. Hayasaka',
                            'number': 22,
                            'photo': 'https://media-4.api-sports.

In [19]:
squad_data_json

{'get': 'players/squads',
 'parameters': {'team': '294'},
 'errors': [],
 'results': 1,
 'paging': {'current': 1, 'total': 1},
 'response': [{'team': {'id': 294,
    'name': 'Kawasaki Frontale',
    'logo': 'https://media-4.api-sports.io/football/teams/294.png'},
   'players': [{'id': 32946,
     'name': 'Jung Sung-Ryong',
     'age': 38,
     'number': 1,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/32946.png'},
    {'id': 32943,
     'name': 'S. Ando',
     'age': 33,
     'number': 21,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/32943.png'},
    {'id': 315353,
     'name': 'Y. Hayasaka',
     'age': 24,
     'number': 22,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/football/players/315353.png'},
    {'id': 33785,
     'name': 'N. Kamifukumoto',
     'age': 34,
     'number': 99,
     'position': 'Goalkeeper',
     'photo': 'https://media-4.api-sports.io/fo

In [27]:
# Code to quickly rename files with certain pattern
# This will save time, since we have to download from remove.bg and they append text to name
# After we dump into image directory, run this to rename, and avoid manually changing hassle
import os

def rename_files_with_text(dir_path, old_text, new_text):
    for filename in os.listdir(dir_path):
        if old_text in filename:
            new_filename = filename.replace(old_text, new_text)
            os.rename(os.path.join(dir_path, filename), os.path.join(dir_path, new_filename))

dir_path = r'C:\Users\lover\OneDrive\Documents\Sports Interactive\Football Manager 2024\graphics\faces\custom_faces'
old_text = "-removebg-preview"
new_text = ""
rename_files_with_text(dir_path, old_text, new_text)
                      

{'get': 'teams',
 'parameters': {'league': '100', 'season': '2023'},
 'errors': [],
 'results': 20,
 'paging': {'current': 1, 'total': 1},
 'response': [{'team': {'id': 297,
    'name': 'FC Gifu',
    'code': 'GIF',
    'country': 'Japan',
    'founded': 2001,
    'national': False,
    'logo': 'https://media-4.api-sports.io/football/teams/297.png'},
   'venue': {'id': 958,
    'name': 'Nagaragawa Stadium',
    'address': 'Gifu-ken',
    'city': 'Gifu',
    'capacity': 31000,
    'surface': 'grass',
    'image': 'https://media-4.api-sports.io/football/venues/958.png'}},
  {'team': {'id': 304,
    'name': 'Matsumoto Yamaga',
    'code': 'MAT',
    'country': 'Japan',
    'founded': 1965,
    'national': False,
    'logo': 'https://media-4.api-sports.io/football/teams/304.png'},
   'venue': {'id': 969,
    'name': 'Sunpro Alwin Stadium',
    'address': 'Nagano 5300',
    'city': 'Matsumoto',
    'capacity': 20336,
    'surface': 'grass',
    'image': 'https://media-4.api-sports.io/footba

{'get': 'teams',
 'parameters': {'league': '497', 'season': '2023'},
 'errors': [],
 'results': 15,
 'paging': {'current': 1, 'total': 1},
 'response': [{'team': {'id': 7117,
    'name': 'Biwako Shiga',
    'code': None,
    'country': 'Japan',
    'founded': 2005,
    'national': False,
    'logo': 'https://media-4.api-sports.io/football/teams/7117.png'},
   'venue': {'id': 5182,
    'name': 'Nunobiki Higashiomi Stadium',
    'address': '1503 Shibahara-cho',
    'city': 'Higashiomi',
    'capacity': 5000,
    'surface': 'grass',
    'image': 'https://media-4.api-sports.io/football/venues/5182.png'}},
  {'team': {'id': 7123,
    'name': 'Honda',
    'code': None,
    'country': 'Japan',
    'founded': 1971,
    'national': False,
    'logo': 'https://media-4.api-sports.io/football/teams/7123.png'},
   'venue': {'id': 5185,
    'name': 'Honda Miyakoda Soccer Stadium',
    'address': 'Toda-cho, Kita-ku',
    'city': 'Hamamatsu',
    'capacity': 4000,
    'surface': 'grass',
    'image': 

{'errors': [],
 'get': 'players/squads',
 'paging': {'current': 1, 'total': 1},
 'parameters': {'team': '285'},
 'response': [{'players': [{'age': 30,
                            'id': 33753,
                            'name': 'M. Tomizawa',
                            'number': 1,
                            'photo': 'https://media-4.api-sports.io/football/players/33753.png',
                            'position': 'Goalkeeper'},
                           {'age': 25,
                            'id': 32848,
                            'name': 'G. Hatano',
                            'number': 21,
                            'photo': 'https://media-4.api-sports.io/football/players/32848.png',
                            'position': 'Goalkeeper'},
                           {'age': 23,
                            'id': 318822,
                            'name': 'L. Radotic',
                            'number': 30,
                            'photo': 'https://media-4.api-sports.io/

In [24]:
def get_team_data(team_id):
    import json
    import http.client
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")
    api_key = os.getenv('x-rapidapi-key')
    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': api_key
    }
    team_id = str(team_id)
    request_string = f"/players/squads?team={team_id}"
    #conn.request("GET", "/players/squads?team=294", headers=headers)
    conn.request("GET", request_string, headers=headers)

    res = conn.getresponse()
    return json.load(res)
    

In [25]:
kataller_toyama_json = get_team_data(4322)
pprint.pprint(kataller_toyama_json)

{'errors': [],
 'get': 'players/squads',
 'paging': {'current': 1, 'total': 1},
 'parameters': {'team': '4322'},
 'response': [{'players': [{'age': 41,
                            'id': 33786,
                            'name': 'T. Shibasaki',
                            'number': 1,
                            'photo': 'https://media-4.api-sports.io/football/players/33786.png',
                            'position': 'Goalkeeper'},
                           {'age': 21,
                            'id': 268608,
                            'name': 'T. Tagawa',
                            'number': 21,
                            'photo': 'https://media-4.api-sports.io/football/players/268608.png',
                            'position': 'Goalkeeper'},
                           {'age': 27,
                            'id': 109346,
                            'name': 'K. Saito',
                            'number': 31,
                            'photo': 'https://media-4.api-sports.i

In [26]:
# Monteido Yamagata 312
monteido_yamagata_json = get_team_data(312)
pprint.pprint(monteido_yamagata_json)

{'errors': [],
 'get': 'players/squads',
 'paging': {'current': 1, 'total': 1},
 'parameters': {'team': '312'},
 'response': [{'players': [{'age': 29,
                            'id': 33533,
                            'name': 'M. Goto',
                            'number': 1,
                            'photo': 'https://media-4.api-sports.io/football/players/33533.png',
                            'position': 'Goalkeeper'},
                           {'age': 28,
                            'id': 33784,
                            'name': 'K. Hasegawa',
                            'number': 16,
                            'photo': 'https://media-4.api-sports.io/football/players/33784.png',
                            'position': 'Goalkeeper'},
                           {'age': 31,
                            'id': 32945,
                            'name': 'E. Fujishima',
                            'number': 23,
                            'photo': 'https://media-4.api-sports.io/f

In [17]:
# List of vars
unmatched_players
j2_league_teams_2023_json
j3_league_teams_2023_json
jfl_league_teams_2023_json
kawasaki_frontale
v_varen_team_json
kataller_toyama_json
monteido_yamagata_json

{'2000061718': {'name': 'Kaito Kamiya', 'position': '-'},
 '2000151923': {'name': 'Renji Matsui', 'position': '-'},
 '45111889': {'name': 'Koki Harada', 'position': '-'},
 '2000238714': {'name': 'Takatora Einaga', 'position': ''},
 '2000151925': {'name': 'Taiyo Igarashi', 'position': ''}}

In [68]:
# Open shelve object
with shelve.open('mydata') as db:
    # Add variables to shelve object
    #db['unmatched_players'] = unmatched_players
    db['j2_league_teams_2023_json'] = j2_league_teams_2023_json
    db['j3_league_teams_2023_json'] = j3_league_teams_2023_json
    db['jfl_league_teams_2023_json'] = jfl_league_teams_2023_json
    #db['kawasaki_frontale'] = kawasaki_frontale
    db['v_varen_team_json'] = v_varen_team_json
    db['kataller_toyama_json'] = kataller_toyama_json
    db['monteido_yamagata_json'] = monteido_yamagata_json

In [69]:
# Convenience function to save variables
def save_to_shelve(var, db_name='mydata'):
    # Get the name of the variable
    var_name = None
    for name, value in globals().items():
        if value is var:
            var_name = name
            break

    # Open shelve object
    with shelve.open(db_name) as db:
        # Add variable to shelve object
        db[var_name] = var

In [64]:
test_var = 'I made your brother become a boy scout'
#pprint.pprint(locals().items())

# var_name = None
# for name, value in globals().items():
#     if value is test_var:
#         var_name = name
#         break
# print(f"Our var name is {var_name}")

save_to_shelve(test_var)

In [65]:
# Open shelved variable
with shelve.open('mydata') as db:
    new_test_var = db['test_var']
    
print(f"Our new test var is {test_var}")

Our new test var is I made your brother become a boy scout


In [67]:
# Retrieve list of all variables
with shelve.open('mydata') as db:
    # Get list of keys
    keys = list(db.keys())

# Print list of keys
print(keys)

['j2_league_teams_2023_json', 'j3_league_teams_2023_json', 'jfl_league_teams_2023_json', 'v_varen_team_json', 'kataller_toyama_json', 'monteido_yamagata_json', 'var', 'test_var']


In [34]:
def generate_player_images(html_file, squad_data_json, image_output_dir=IMAGE_DIR):
    # Create an images directory if none already exists
    if not os.path.exists(image_output_dir):
        os.makedirs(image_output_dir)

    def download_image(image_url: str, uid: str) -> bool:
        local_image_file = f'{image_output_dir}/{uid}.png'

        if Path(local_image_file).exists():
            print(f'The file {local_image_file} exists, will not download')
            return False

        response = requests.get(image_url)
        if response.status_code == 200:
            # Save the image with UID as the filename
            with open(local_image_file, 'wb') as image_file:
                image_file.write(response.content)
            return True  # Return True to indicate success
        else:
            return False  # Return False to indicate failure

    # Parse the HTML file
    with open(html_file, 'r', encoding='utf-8') as html_file:
        soup = BeautifulSoup(html_file, 'html.parser')

    # Extract player information from HTML
    player_data_html = []
    for row in soup.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        if len(columns) >= 4:
            position = columns[0].get_text()
            name = columns[2].get_text()
            uid = columns[3].get_text()
            age = columns[7].get_text()
            player_data_html.append({'position': position, 'name': name, 'uid': uid, 'age': age})

    # Create a dictionary to store the matched players and image URLs
    matched_players = {}
    unmatched_players = {}
    # Iterate over player data from HTML
    for player_html in player_data_html:
        # Define a threshold for fuzzy matching score
        # We HAD to lower this to 70 to match even simple, non-duplicated names. 
        matching_threshold = 70

        # Initialize variables for best matching player and image URL
        best_match = None
        best_match_score = 0
        best_image_url = None

        # Iterate over players in JSON data
        for player_json in squad_data_json['response'][0]['players']:
            # Calculate fuzzy matching score between HTML and JSON player names
            name_similarity = fuzz.partial_ratio(player_html['name'], player_json['name'])

            # Check if the player has the same position and similar age
            position_match = player_html['position'] == player_json['position']
            age_match = player_html['age'] == str(player_json['age'])

            # Calculate a weighted score based on name similarity, position, and age match
            total_score = (name_similarity * 0.6) + (position_match * 0.2) + (age_match * 0.2)

            # Update best match if the current player is a better match
            if total_score > best_match_score and name_similarity >= matching_threshold:
                best_match = player_json
                best_match_score = total_score

        # If a match is found, extract the image URL and download the image
        if best_match:
            best_image_url = best_match['photo']
            if download_image(best_image_url, player_html['uid']):
                # Download was successful, add player to matched players
                matched_players[player_html['uid']] = {'player_data': best_match, 'image_url': best_image_url}

                # Add a random time delay between 15 and 60 seconds
                delay_seconds = random.randint(15, 60)
                time.sleep(delay_seconds)

                # Continue with additional processing or actions
                print(f"Downloaded image for player with UID: {player_html['uid']}")
        else:
            unmatched_players[player_html['uid']] = {'name': player_html['name'], 'position': player_html['position']}



In [31]:
yokohama_f_json = get_team_data(296)
# generate_player_images('yokohama_f.html', yokohama_f_json, 'yokohama_f_images')

In [39]:
generate_player_images('yokohama_f.html', yokohama_f_json, 'yokohama_f_images')

Downloaded image for player with UID: 45088078
Downloaded image for player with UID: 784212
Downloaded image for player with UID: 45114800
Downloaded image for player with UID: 19139361
Downloaded image for player with UID: 45065336
Downloaded image for player with UID: 45113971
Downloaded image for player with UID: 45107070
Downloaded image for player with UID: 2000062147
Downloaded image for player with UID: 45030041
Downloaded image for player with UID: 45031033
Downloaded image for player with UID: 2000164947
Downloaded image for player with UID: 2000065730
Downloaded image for player with UID: 45086555
Downloaded image for player with UID: 45083353
Downloaded image for player with UID: 2000063093
Downloaded image for player with UID: 45060234
Downloaded image for player with UID: 45104514
Downloaded image for player with UID: 790435
Downloaded image for player with UID: 45121840
Downloaded image for player with UID: 45107198
Downloaded image for player with UID: 19122788
Downloade